# 第三章 提示优化工具

<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一、引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
     <li>
         <span><a href="#二、优化 Prompt 的代码示例" data-toc-modified-id="二、优化 Prompt 的代码示例">二、优化 Prompt 的代码示例</a></span>
         <ul class="toc-item">
             <li><span><a href="##2.1 示例 1" data-toc-modified-id="2.1 示例 1">2.1 示例 1</a></span></li>
             <li><span><a href="##2.2 示例 2" data-toc-modified-id="2.2 示例 2">2.2 示例 2</a></span></li>
             <li><span><a href="##2.3 示例 3" data-toc-modified-id="2.3 示例 3">2.3 示例 3</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三、总结与讨论" data-toc-modified-id="三、总结与讨论">三、总结与讨论</a></span></li>
     </ul>
</div>

## 一、引言
***

对于大语言模型（Large Language Models, LLMs），**我们通常需要遵循一些原则和策略来创建能产生高质量回答的有效提示：**

1. 明确和具体：你应该尽可能明确和具体地描述你希望 LLM 执行的任务。如果你期望得到特定类型的回答，应在提示中明确指出。同时，如果存在任何具体的限制或要求，也应明确提出。

2. 开放式与封闭式：你可以根据需求选择开放式问题（允许多种回答）或封闭式问题（限制回答范围）。这两种类型的问题各有优势，你应根据实际需求进行选择。

3. 语境清晰：为了让 LLM 生成更有意义的回答，你需要确保提供足够的上下文。如果提示是基于之前的信息，那么这些信息应被包含在内。

4. 创造力和想象力：如果你期望得到创新性的输出，可以鼓励人工智能进行发散思维或头脑风暴。如果这符合你的需求，你还可以建议人工智能模拟某些场景。

**实际上，我们可以利用这些原则和策略来有效地优化提示。**

本章将介绍一个适用于 LLM 的优化提示工具。这个工具能够帮助你像 [PromptPerfect](https://promptperfect.jina.ai/home) 那样有效地优化提示，从而最大限度地发挥 LLM 的潜力，并获得更准确、与上下文相关的回答。


## 二、优化 Prompt 的代码示例
***

注意：我使用的测试环境是 Anaconda + Python 3.8.10。Python 的版本要求 Python 3.7+。

开始实验之前，我们需要安装好本次实验所需的第三方依赖库：`openai`：OpenAI Python 库提供了一个便捷的途径，让任何 Python 3.7+ 应用程序可以访问 OpenAI REST API。你可以通过运行以下命令来安装：`pip install openai==1.10.0`。

导入此次实验所需的依赖库

In [1]:
import os
from openai import OpenAI


# 从环境变量中获取 OpenAI API Key 或者直接赋值
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_API_KEY = "sk-...你的 OpenAI API Key"

# 如果您使用的是官方 API，就直接用 https://api.openai.com/v1 就行。
BASE_URL = "https://api.openai.com/v1"
# 如果您使用的不是官方 API，而是通过代理进行请求，请设置您的代理 URL。
# BASE_URL = "https://api.xxx.../v1"

In [2]:
# 实例化 OpenAI 对象
# 传入参数：OpenAI API Key（必需）、Base URL 和最大重试次数
client = OpenAI(api_key=OPENAI_API_KEY, base_url=BASE_URL, max_retries=3)

In [3]:
# 参数 n，整数或 Null，可选项，默认为 1。为每条输入信息生成多少个聊天完成选项。
# 参数 temperature，实数值或 Null，可选项，默认为 1。使用的采样温度，介于 0 和 2 之间。0.8 等较高值会使输出更加随机，而 0.2 等较低值会使输出更加集中和确定。

def get_completions1(system_instruction, llm_prompt, model_endpoint):

    response = client.chat.completions.create(model=model_endpoint,
                                              messages=[{"role": "system", "content": system_instruction},
                                                        {"role": "user", "content": llm_prompt}
                                                       ],
                                              n=1, temperature=0.60, seed=42,
                                              presence_penalty=0, frequency_penalty=0,
                                              max_tokens=1024
                                             )

    return response.choices[0].message.content.strip()


def get_completions2(system_instruction, llm_prompt, model_endpoint):

    response = client.chat.completions.create(model=model_endpoint,
                                              messages=[{"role": "system", "content": system_instruction},
                                                        {"role": "user", "content": llm_prompt}
                                                       ],
                                              n=1, temperature=0, seed=42,
                                              presence_penalty=0, frequency_penalty=0,
                                              max_tokens=1024
                                             )

    return response.choices[0].message.content.strip()

下面给出的 `system_instruction` 模板的本质是利用**上下文学习**（Few-shot 演示）和提示工程。

In [4]:
# 中文
system_instruction1 = f"""你是一位出色的人工智能提示工程师。你是设计 ChatGPT 提示的专家，能取得最佳效果。

要创建能产生高质量回答的有效提示，请考虑以下原则和策略：
1. 明确和具体：尽可能明确、具体地说明你希望人工智能做些什么。如果你想要某种类型的答复，请在提示中简要说明。如果有具体的限制或要求，也一定要写明。
2. 开放式与封闭式：根据你的需求，你可能会选择开放式问题（允许多种回答）或封闭式问题（缩小可能回答的范围）。两者都有其用途，请根据自己的需要进行选择。
3. 语境清晰：确保提供足够的上下文，以便人工智能生成有意义的相关回复。如果提示是基于先前的信息，则应确保包含这些信息。
4. 创造力和想象力：如果你想要创造性的输出，可以鼓励人工智能发散思维或进行头脑风暴。如果符合你的需求，你甚至可以建议人工智能想象某些场景。

你的任务是根据用户给定的提示进行优化，设计出一个新的、经过优化的 ChatGPT 提示。

下面提供了一些好的示例，供你参考：
原始提示：你的任务是使用 Python，根据用户给的数据绘制散点图。
优化后的提示：你的任务是使用 Python 编程语言，根据用户给定的数据绘制一个散点图。请确保散点图能够清晰地展示数据之间的关系，并使用适当的颜色、形状和标签来区分不同的数据点。此外，你还需要为图表添加标题、坐标轴标签以及图例（如果适用），以便更好地解释数据的含义。最后，请将生成的散点图保存为一个图像文件，以便用户可以查看和分享。
原始提示：你的任务是将用户给定的英文翻译成中文。
优化后的提示：请将以下英文文本准确地翻译成中文，保持原文的意思和语境不变。如果英文中包含专有名词或术语，请在翻译时注明其标准中文名称或提供相应的解释。此外，请确保翻译后的中文句子通顺、语法正确，并且符合中文的表达习惯。
原始提示：你的任务是指导初学者系统地学习机器学习。
优化后的提示：你被要求为一个完全没有机器学习背景的初学者设计一个详细的学习计划。请你描述这个计划，包括以下维度：首先，推荐适合初学者的机器学习入门教材或在线课程；其次，提供一个结构化的学习路线图，明确指出在学习过程中应该先掌握哪些基本概念，再逐步深入到哪些高级主题；最后，请提出一些实践建议，比如初学者可以通过哪些项目来巩固理论知识，以及在遇到困难时如何寻求帮助和资源。请确保你的回答涵盖了机器学习的基础知识、算法理解、编程实践，以及如何解决实际问题等方面。
"""

# 英文
system_instruction2 = f"""You are an exceptional AI prompt engineer. \
You are an expert at writing ChatGPT Prompts for best results.

To create efficient prompts that yield high-quality responses, \
consider the following principles and strategies:
1. Clear and Specific: Be as clear and specific as possible about what you want from the AI. If you want a certain type of response, outline that in your prompt. If there are specific constraints or requirements, make sure to include those as well.
2. Open-ended vs. Closed-ended: Depending on what you're seeking, you might choose to ask an open-ended question (which allows for a wide range of responses) or a closed-ended question (which narrows down the possible responses). Both have their uses, so choose according to your needs.
3. Contextual Clarity: Make sure to provide enough context so that the AI can generate a meaningful and relevant response. If the prompt is based on prior information, ensure that this is included.
4. Creativity and Imagination: If you want creative output, encourage the AI to think outside the box or to brainstorm. You can even suggest the AI to imagine certain scenarios if it fits your needs.

Your task is to design a new, optimized ChatGPT prompt based on the user-given prompt.
Some good examples are provided below for your reference:
Original prompt: Your task is to plot a scatterplot based on data given by the user, using Python.
Optimized prompt: Your task is to use the Python programming language to draw a scatterplot based on data given by a user. Make sure that the scatterplot clearly shows the relationships between the data and uses appropriate colors, shapes, and labels to distinguish the different data points. In addition, you will need to add a title, axis labels, and a legend (if applicable) to the chart to better explain the meaning of the data. Finally, please save the resulting scatterplot as an image file so that users can view and share it.
Original prompt: Your task is to translate the English text given by the user into Chinese.
Optimized prompt: Please translate the following English text accurately into Chinese, keeping the meaning and context of the original text intact. If the English text contains proper nouns or terms, please indicate their standard Chinese names or provide corresponding explanations when translating. In addition, please make sure that the translated Chinese sentences are fluent, grammatically correct, and in line with Chinese expressions.
Original prompt: You have been tasked with guiding a beginner to learn machine learning in a systematic way.
Optimized prompt: You have been asked to design a detailed learning plan for a beginner who has no machine learning background at all. Please describe the plan, including the following dimensions: first, recommend introductory machine learning textbooks or online courses suitable for beginners; second, provide a structured learning roadmap that clearly indicates which basic concepts should be mastered before progressively delving into which advanced topics during the learning process; finally, please make some practical suggestions, such as which projects beginners can work on to consolidate their theoretical knowledge and how to seek help and resources when they are how to seek help and resources when encountering difficulties. Please make sure your answer covers the basics of machine learning, algorithmic understanding, programming practices, and how to solve real-world problems.
"""

### 2.1 示例 1

In [5]:
llm = "gpt-3.5-turbo-0125"

original_prompt = "你的任务是使用 Python，根据用户给的数据绘制热力图。"
original_prompt = f"""原始提示：{original_prompt}"""

optimized_prompt = get_completions1(system_instruction1, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "你是世界知识专家。"
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

原始提示：你的任务是使用 Python，根据用户给的数据绘制热力图。

优化后的提示：你需要使用 Python 编程语言，根据用户提供的数据生成一个热力图。确保热力图清晰地展示数据之间的关系，使用适当的颜色映射和标签来呈现数据密度和分布情况。请为热力图添加标题、坐标轴标签以及必要的图例，以便用户能够准确理解数据的含义。最后，将生成的热力图保存为图像文件，以便用户查看和分享。

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
要绘制热力图，你可以使用 Python 中的 Matplotlib 库和 Seaborn 库。首先，确保你已经安装了这两个库。然后，你可以按照以下步骤使用用户提供的数据绘制热力图：

1. 导入必要的库：

```python
import matplotlib.pyplot as plt
import seaborn as sns
```

2. 准备数据。假设用户提供的数据是一个二维数组 `data`：

```python
data = [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]
```

3. 使用 Seaborn 库绘制热力图：

```python
sns.heatmap(data, annot=True, cmap='coolwarm')
plt.show()
```

这段代码将绘制一个热力图，数据来自于 `data`，并且显示每个单元格的数值。你可以根据实际情况调整数据和热力图的样式。

----------------------------------------------------------------------------------------------------
优化后的提示的生成结果：
为了生成热力图，你可以使用 Python 中的 Matplotlib 和 Seaborn 库。以下是一个简单的示例代码，演示如何生成一个基本的热力图：

```python
import numpy as np
import seaborn as sns
import matplotlib.p

In [6]:
llm = "gpt-3.5-turbo-0125"

# 英文
original_prompt = "Your task is to use Python and draw a heat map based on the data given by the user."
original_prompt = f"""Original prompt: {original_prompt}"""

optimized_prompt = get_completions1(system_instruction2, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "You are an expert at world knowledge."
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

Original prompt: Your task is to use Python and draw a heat map based on the data given by the user.

Optimized prompt: Your task is to create a heatmap using Python, utilizing the data provided by the user. Ensure that the heatmap is visually clear and informative, with appropriate color gradients to represent the data variations effectively. Additionally, include axis labels, a title, and a color bar legend to aid in interpreting the heatmap accurately. Finally, save the heatmap as an image file for easy sharing and viewing.

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
To draw a heat map in Python based on user-provided data, you can use libraries such as Matplotlib and Seaborn. Here is an example code snippet that demonstrates how to create a simple heat map using Matplotlib and Seaborn:

```python
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# User-provided data (example data)
data = np

### 2.2 示例 2

In [8]:
llm = "gpt-3.5-turbo-0125"

original_prompt = "你的任务是将用户给定的中文翻译成英文。"
original_prompt = f"""原始提示：{original_prompt}"""

optimized_prompt = get_completions1(system_instruction1, original_prompt, llm)

instruction_ = "你是世界知识专家。"
content = """《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。\
它是一本面向有一定编程基础或实际项目（不一定是算法）经历，\
对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，\
旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。"""
original_prompt = original_prompt + "\n" + f"<text>{content}</text>"
optimized_prompt = optimized_prompt + "\n" + f"<text>{content}</text>"
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

原始提示：你的任务是将用户给定的中文翻译成英文。
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。</text>

优化后的提示：请准确翻译以下中文文本为英文，确保保持原文意思和语境的准确性。如果中文中包含专有名词或术语，请在翻译时注明其标准英文名称或提供相应的解释。此外，请确保翻译后的英文句子通顺、语法正确，并符合英文表达习惯。
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。</text>

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
"The Butterfly Book" aims to help reduce barriers, bridge the gap between applications and research, and make the development of large-scale model applications more accessible. It is a book targeted at learners with some programming background or practical project experience (not necessarily in algorithms) who are interested in ChatGPT (or similar models) and want

In [9]:
llm = "gpt-3.5-turbo-0125"

original_prompt = "Your task is to translate the Chinese given by the user into English."
original_prompt = f"""Original prompt: {original_prompt}"""

optimized_prompt = get_completions2(system_instruction2, original_prompt, llm)

instruction_ = "You are an expert at world knowledge."
content = """《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。\
它是一本面向有一定编程基础或实际项目（不一定是算法）经历，\
对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，\
旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。"""
original_prompt = original_prompt + "\n" + f"<text>{content}</text>"
optimized_prompt = optimized_prompt + "\n" + f"<text>{content}</text>"
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

Original prompt: Your task is to translate the Chinese given by the user into English.
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。</text>

Optimized prompt: Your task is to accurately translate the Chinese text provided by the user into English while ensuring that the meaning and context of the original text are preserved. Please make sure that the translated English sentences are fluent, grammatically correct, and effectively convey the intended message of the Chinese text. Additionally, if there are any specific cultural references or nuances in the Chinese text that need to be explained or adapted for an English-speaking audience, please include those in the translation.
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地

### 2.3 示例 3

In [10]:
llm = "gpt-3.5-turbo-0125"

original_prompt = "你的任务是指导初学者系统地学习数据科学。"
original_prompt = f"""原始提示：{original_prompt}"""

optimized_prompt = get_completions1(system_instruction1, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "你是世界知识专家。"
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

原始提示：你的任务是指导初学者系统地学习数据科学。

优化后的提示：你被要求为一个完全没有数据科学背景的初学者设计一个详细的学习计划。请你描述这个计划，包括以下维度：首先，推荐适合初学者的数据科学入门教材或在线课程；其次，提供一个结构化的学习路线图，明确指出在学习过程中应该先掌握哪些基本概念，再逐步深入到哪些高级主题；最后，请提出一些实践建议，比如初学者可以通过哪些项目来巩固理论知识，以及在遇到困难时如何寻求帮助和资源。请确保你的回答涵盖了数据科学的基础知识、数据清洗、数据分析、机器学习等方面。

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
学习数据科学是一个系统性的过程，需要掌握一系列基础知识和技能。以下是我建议的学习路径：

1. 编程基础：学习编程语言如Python或R，掌握基本语法和数据结构。
2. 统计学基础：了解统计学的基本概念，包括概率、假设检验、回归分析等。
3. 数据处理：学习数据清洗、转换和处理技术，包括使用Pandas或dplyr等工具。
4. 数据可视化：掌握数据可视化技术，如Matplotlib、Seaborn或ggplot2。
5. 机器学习：学习机器学习算法和技术，包括监督学习、无监督学习和深度学习。
6. 数据分析：学习如何应用统计和机器学习技术进行数据分析和建模。
7. 实践项目：通过实际项目来应用所学知识，提升实战能力。

建议初学者可以通过在线课程、教科书、博客和实践项目来学习数据科学。同时，参与数据科学社区和论坛，与其他学习者交流经验和解决问题。持续学习和实践是提升数据科学能力的关键。祝你学习顺利！

----------------------------------------------------------------------------------------------------
优化后的提示的生成结果：
学习数据科学对于初学者来说可能是一个挑战，但通过有计划的学习和实践，你可以逐步掌握这门技能。以下是一个详细的学习计划：

### 教材和在线课程推荐：
1. **教材**：
   - "Python for Data Analy

In [11]:
llm = "gpt-3.5-turbo-0125"

original_prompt = "Your task is to guide beginners through the systematic study of data science."
original_prompt = f"""Original prompt: {original_prompt}"""

optimized_prompt = get_completions1(system_instruction2, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "You are an expert at world knowledge."
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

Original prompt: Your task is to guide beginners through the systematic study of data science.

Optimized prompt: You have been tasked with creating a comprehensive guide for beginners who are interested in studying data science systematically. Please outline a detailed roadmap that covers the essential topics and skills they should focus on, starting from the basics and progressing to more advanced concepts. Include recommendations for beginner-friendly resources such as books, online courses, and tutorials. Additionally, suggest practical projects or exercises that can help reinforce their learning and provide tips on how to stay motivated and seek assistance when needed. Make sure to emphasize the importance of hands-on practice, critical thinking, and continuous learning in the field of data science.

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
To guide beginners through the systematic study of data science, it's i

## 三、总结与讨论
***

**优化 ChatGPT 的 Prompts 对于提升交互体验和生成结果的质量非常重要。**以下是一些主要的原因：
- 提高准确性：通过优化 Prompts，我们可以更准确地引导 ChatGPT 生成我们期望的回答。这对于获取准确、相关和有用的信息至关重要。

- 提高效率：优化的 Prompts 可以减少无效或者不相关的回答，从而提高交互的效率。

- 增强可控性：通过优化 Prompts，我们可以更好地控制 ChatGPT 的输出，使其更符合用户的需求和期望。

- 提升用户体验：优化的 Prompts 可以帮助 ChatGPT 更好地理解用户的需求，从而提供更满意的回答，提升用户体验。

总的来说，优化 Prompts 是提高 ChatGPT 性能和用户满意度的关键步骤。它有助于我们更好地利用提示工程（Prompt Engineering），为用户提供高质量的服务。

本章将介绍一个适用于 LLM 的优化提示工具。这个工具能够帮助你像 [PromptPerfect](https://promptperfect.jina.ai/home) 那样有效地优化提示，从而最大限度地发挥 LLM 的潜力，并获得更准确、与上下文相关的回答。

在实际应用中，为 LLM 提供的演示对于优化 Prompt 的性能具有重要意义。对于特定需求，如果手动构建优质的演示（**成对的原始提示和优化后的提示**）存在困难，我们可以利用一些商业化（非开源）的提示优化工具，如：[PromptPerfect](https://promptperfect.jina.ai/home)、[百度千帆大模型操作台的 Prompt 优化](https://cloud.baidu.com/doc/WENXINWORKSHOP/s/Clommng91)、[讯飞星火的指令优化](https://xinghuo.xfyun.cn/desk)等。这些商业工具通常进行了大量的工程优化，能够适应各种场景，满足用户的特定需求。
